# Mitiq Zero Noise Extrapolation Notebook


This notebook serves as a guide to use every feature of the Mitiq Zero Noise Extrapolation (ZNE) parameter optimizer. 

We present 3 optimization methods, which can all be edited by the user and easily adapted to their own needs. We include 6 noise models and 7 example circuits as a proof of concept, but the purpose of this ZNE optimizer is for the user to input the following: a backend, a circuit, a function to check how many circuit runs were succesful, and the ideal result of the circuit. We provide easy "plug and play" functions to optimize over any set of backends and circuits, along with some examples for simplification. 


Run the following code block to import all necessary libraries:

In [2]:
from experiment import make_executor
import experiment
from routines import light_brute_force_search, brute_force_search, adaptive_search
from noise_model_backends import get_noise_backend
from circuits import get_experiment

Set the following to "True" to silence some unnecessary warnings throughout the notebook: 

In [3]:
experiment.SILENCE_WARNINGS=True 

### The following two blocks of code allow you to input your desired noise model and circuits. The ZNE optimizer will find the optimal parameters for with respect to the given input. 

In order to select the desired noise model and circuit, simply copy the name from the list into the variable in the code like in the given example. 

We include the following noise models: 

{depolarizing, amplitude_damping, phase_damping, readout, thermal, general_zne}

You can also input any desired noise model, including the backend for a noise model based on a real quantum chip, say "ibm_fez"). If you have your own noise model you want to input, ignore this variable for now*.


In [4]:
NOISE_MODEL="general_zne"

This next block of code allows you to select the circuits. We include the following circuits:

{ghz, mirror_circuits, rb_circuits, rotated_rb_circuits, random_clifford_t, w_state, qpe}


In [5]:
CIRCUIT="ghz"

The following block of code creates our set of test "batch" experiments, which are optional (and set at the default). These can reduce the search space for our optimizer. This is meant for the user to edit only if they know which reduced space they want searched.

In [6]:

zne_batch_test = {
    "noise_scaling_factors": [[1, 3, 5, 7], [1, 3, 5]],  # Noise scaling values
    "noise_scaling_method": ["global", "local_random", "local_all", "identity_scaling"],
    "extrapolation": ["linear", "richardson", "polynomial", "exponential", "poly-exp", "adaptive-exp"],
}

The following block of code creates the circuit and verifying function of the experiment. 

In [7]:

circ, verify_func, ideal_result= get_experiment(CIRCUIT)

The following block of code creates the backend for the experiment. Default parameters are used unless instructed otherwise. 
As mentioned previously, if you would like to enter your own* backend and noise models, this can be done here. 

In [8]:
backend = get_noise_backend(NOISE_MODEL)
#backend=get_noise_backend(NOISE_MODEL, prob=0.005, param_amp=0.9, excited_state_population=0, param_phase=0.9, canonical_kraus=True)

# Or enter your own backend here:

The following block makes the executor. This is also not meant to be edited and must be run. 

In [9]:
exe=make_executor(backend, verify_func, shots=4096)

### Optimization methods
Now that the setup is complete, we can move on to the optimization methods. 

#### Adaptive search 
The first method is the adaptive search. Default parameters are in place, and zne_batch_test can be added as a parameter in order to reduce the search space based on the user's changes to zne_batch_test. Both options are presented. 

Optional:
We also present the possibility of changing some optimization parameters, such as the maximum number of scale factors and the tolerance. 

tolerance: if the improvement is lower that the tolerance, stop expanding the list.

In [10]:

adaptive_search(circ , exe, ideal_result)
# adaptive_search(circ , exe, ideal_result, zne_batch_test)
# max_factors=10
# tolerance=1e-4
# adaptive_search(circ , exe, ideal_result, tolerance, max_factors)

KeyboardInterrupt: 

#### Light brute force

The second method is a lighter version of brute force. As for the previous method, the search space can be reduced by adding zne_batch_test as a parameter here as well. 
We present another optional parameter here: max_iter. This is just a stop criteria in the case of getting stuck in an infinite loop. 

In [ ]:
# max_iter=10 # optional

light_brute_force_search(circ , exe, ideal_result)
# light_brute_force_search(circ , exe, ideal_result, zne_batch_test, max_iter)


#### Brute force 

The last method is a brute force method. This method consumes the most resources but will output the best parameters for ZNE. zne_batch_test can be added as a parameter if the user knows how to reduce the search space themselves. 

In [ ]:

brute_force_search(circ, exe, ideal_result)
# brute_force_search(circ, exe, ideal_result, zne_batch_test)